# **MD**$\chi$ - Hypertune

Trading with triple-barrier approach.

In [2]:
from src.trading_data import get_data
from src.triple_barrier import triple_barrier_labels
from src.temporian_prep import temporian_eventset
from src.train_model import train
from src.prediction import create_predictions
from src.trader import execute_strategy

from datetime import datetime
import pandas as pd

2024-02-17 15:09:50.899273: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-17 15:09:51.696193: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-17 15:09:51.696317: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-17 15:09:51.861612: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-17 15:09:52.209554: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-17 15:09:52.213814: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [3]:
def mdx(
    df: pd.DataFrame,
    barrier_length: int,
    barrier_std: float,
    days_lookback: int,
    split_date: str,
    model_summary: str,
    verbose: int,
    lower_prob: float,
    upper_prob: float
):
        
    # Calculate triple barrier
    labels = triple_barrier_labels(
        df=df['close'],
        t=barrier_length,
        devs=barrier_std
    )
    
    # Join tbl with df
    df = df.join(labels)
    df.dropna(inplace=True)
    
    # One hot encode labels
    one_hot_encoded = pd.get_dummies(df['Label'], prefix='label')

    # Concatenate the one-hot encoded columns to the original DataFrame
    df = pd.concat([df, one_hot_encoded], axis=1)
    del df['Label']
    
    (feature_names, label_names,
     test_dataset,
     train_dataset,
     tf_test_dataset,
     tf_train_dataset) = temporian_eventset(
        df=df,
        days_lookback=days_lookback,
        split_date=split_date
    )
    # Train model
    model = train(
        train_dataset=tf_train_dataset,
        label_names=label_names,
        model_summary=model_summary,
        verbose=verbose
    )
    
    # Create predictions
    result_df = create_predictions(
        tf_test_dataset=tf_test_dataset,
        model=model,
        pandas_test_dataset=test_dataset
    )
    
    # Execute trading
    overall_profit, n_trades = execute_strategy(
        data=result_df,
        lower_prob=lower_prob,
        upper_prob=upper_prob
    )
    
    return overall_profit, n_trades

In [ ]:
# Hyperparameters

start='2020-01-01'
end='2022-12-31'
split_date=datetime(2022, 1, 1)
interval_yf='1d'
symbols='BTC-USD'
model_summary=False
verbose=0
BARRIER = [0.2, 0.3, 0.4, 0.5]
UPPER_PROB = [0.4, 0.6, 0.8]
LOWER_PROB = [0.4, 0.6, 0.8]
DAYS_LOOKBACK=30
T_BARRIER =[2, 3, 4, 5, 6, 7, 8 , 9, 10]

hps_results = {
    't_barrier': [],
    'barrier': [],
    'overall_profit': [],
    'n_trades': [],
    'lower_prob': [],
    'upper_prob': []
}

# Get data
df = get_data(
    symbols=[symbols],
    start=start,
    end=end,
    interval=interval_yf
)

for t_barrier in T_BARRIER:
    for barrier in BARRIER:
        for x in LOWER_PROB:
            for z in UPPER_PROB:
                df_2 = df.copy(deep=True)
                overall_profit, n_trades = mdx(
                    df=df_2,
                    barrier_length=t_barrier,
                    barrier_std=barrier,
                    days_lookback=3,
                    split_date=split_date,
                    model_summary=model_summary,
                    verbose=verbose,
                    lower_prob=x,
                    upper_prob=z
                )
                
                hps_results['t_barrier'].append(t_barrier)
                hps_results['barrier'].append(barrier)
                hps_results['overall_profit'].append(overall_profit)
                hps_results['n_trades'].append(n_trades)
                hps_results['lower_prob'].append(x)
                hps_results['upper_prob'].append(z)
                hp_results = pd.DataFrame(hps_results).sort_values('overall_profit', ascending=False)
                print(hp_results.head())

[INFO 24-02-17 15:10:38.7780 UTC kernel.cc:1233] Loading model from path /var/tmp/tmpni_5b0it/model/ with prefix 4f725862cc16467c_0
[INFO 24-02-17 15:10:38.7843 UTC quick_scorer_extended.cc:903] The binary was compiled without AVX2 support, but your CPU supports it. Enable it for faster model inference.
[INFO 24-02-17 15:10:38.7848 UTC abstract_model.cc:1344] Engine "GradientBoostedTreesQuickScorerExtended" built
[INFO 24-02-17 15:10:38.7849 UTC kernel.cc:1061] Use fast generic engine
[INFO 24-02-17 15:10:38.7919 UTC kernel.cc:1233] Loading model from path /var/tmp/tmpni_5b0it/model/ with prefix 4f725862cc16467c_1
[INFO 24-02-17 15:10:38.7935 UTC kernel.cc:1061] Use fast generic engine
[INFO 24-02-17 15:10:38.7993 UTC kernel.cc:1233] Loading model from path /var/tmp/tmpni_5b0it/model/ with prefix 4f725862cc16467c_2
[INFO 24-02-17 15:10:38.8032 UTC kernel.cc:1061] Use fast generic engine


   t_barrier  barrier  overall_profit  n_trades  lower_prob  upper_prob
0          2      0.2     -225.820173       112         0.4         0.4


[INFO 24-02-17 15:10:43.1365 UTC kernel.cc:1233] Loading model from path /var/tmp/tmpviq3ivva/model/ with prefix d9418210612448f9_0
[INFO 24-02-17 15:10:43.1420 UTC kernel.cc:1061] Use fast generic engine
[INFO 24-02-17 15:10:43.1487 UTC kernel.cc:1233] Loading model from path /var/tmp/tmpviq3ivva/model/ with prefix d9418210612448f9_1
[INFO 24-02-17 15:10:43.1502 UTC kernel.cc:1061] Use fast generic engine
[INFO 24-02-17 15:10:43.1562 UTC kernel.cc:1233] Loading model from path /var/tmp/tmpviq3ivva/model/ with prefix d9418210612448f9_2
[INFO 24-02-17 15:10:43.1603 UTC kernel.cc:1061] Use fast generic engine


   t_barrier  barrier  overall_profit  n_trades  lower_prob  upper_prob
1          2      0.2     -174.253083       104         0.4         0.6
0          2      0.2     -225.820173       112         0.4         0.4


[INFO 24-02-17 15:10:46.5724 UTC kernel.cc:1233] Loading model from path /var/tmp/tmpd91ve1wj/model/ with prefix 8da97d01358f4595_0
[INFO 24-02-17 15:10:46.5806 UTC kernel.cc:1061] Use fast generic engine
[INFO 24-02-17 15:10:46.5889 UTC kernel.cc:1233] Loading model from path /var/tmp/tmpd91ve1wj/model/ with prefix 8da97d01358f4595_1
[INFO 24-02-17 15:10:46.5911 UTC kernel.cc:1061] Use fast generic engine
[INFO 24-02-17 15:10:46.5978 UTC kernel.cc:1233] Loading model from path /var/tmp/tmpd91ve1wj/model/ with prefix 8da97d01358f4595_2
[INFO 24-02-17 15:10:46.6022 UTC kernel.cc:1061] Use fast generic engine


...

   t_barrier  barrier  overall_profit  n_trades  lower_prob  upper_prob
7          2      0.2     -125.715668         5         0.8         0.6
6          2      0.2     -128.778325         5         0.8         0.4
2          2      0.2     -134.501958        64         0.4         0.8
8          2      0.2     -134.817987         5         0.8         0.8
1          2      0.2     -174.253083       104         0.4         0.6


[INFO 24-02-17 15:11:19.5026 UTC kernel.cc:1233] Loading model from path /var/tmp/tmpzab4mujq/model/ with prefix 05be9084f93444b8_0
[INFO 24-02-17 15:11:19.5130 UTC kernel.cc:1061] Use fast generic engine
[INFO 24-02-17 15:11:19.5203 UTC kernel.cc:1233] Loading model from path /var/tmp/tmpzab4mujq/model/ with prefix 05be9084f93444b8_1
[INFO 24-02-17 15:11:19.5221 UTC kernel.cc:1061] Use fast generic engine
[INFO 24-02-17 15:11:19.5288 UTC kernel.cc:1233] Loading model from path /var/tmp/tmpzab4mujq/model/ with prefix 05be9084f93444b8_2
[INFO 24-02-17 15:11:19.5310 UTC kernel.cc:1061] Use fast generic engine


    t_barrier  barrier  overall_profit  n_trades  lower_prob  upper_prob
11          2      0.3      -58.517649        19         0.4         0.8
7           2      0.2     -125.715668         5         0.8         0.6
6           2      0.2     -128.778325         5         0.8         0.4
2           2      0.2     -134.501958        64         0.4         0.8
8           2      0.2     -134.817987         5         0.8         0.8


[INFO 24-02-17 15:11:23.3183 UTC kernel.cc:1233] Loading model from path /var/tmp/tmpog8yh6rx/model/ with prefix 7cf7285324ff4fa7_0
[INFO 24-02-17 15:11:23.3291 UTC abstract_model.cc:1344] Engine "GradientBoostedTreesQuickScorerExtended" built
[INFO 24-02-17 15:11:23.3292 UTC kernel.cc:1061] Use fast generic engine
[INFO 24-02-17 15:11:23.3372 UTC kernel.cc:1233] Loading model from path /var/tmp/tmpog8yh6rx/model/ with prefix 7cf7285324ff4fa7_1
[INFO 24-02-17 15:11:23.3389 UTC kernel.cc:1061] Use fast generic engine
[INFO 24-02-17 15:11:23.3451 UTC kernel.cc:1233] Loading model from path /var/tmp/tmpog8yh6rx/model/ with prefix 7cf7285324ff4fa7_2
[INFO 24-02-17 15:11:23.3476 UTC kernel.cc:1061] Use fast generic engine


    t_barrier  barrier  overall_profit  n_trades  lower_prob  upper_prob
11          2      0.3      -58.517649        19         0.4         0.8
7           2      0.2     -125.715668         5         0.8         0.6
6           2      0.2     -128.778325         5         0.8         0.4
2           2      0.2     -134.501958        64         0.4         0.8
8           2      0.2     -134.817987         5         0.8         0.8
